# 코렐라인: 단추 눈의 저택 - Night Phase Test

**"밤"의 가족 회의**: 낮 동안 플레이어가 만진 오브젝트들에 대해 단추 인형 가족이 논의합니다.

- **Phase 1**: 성찰 (Reflection) - NPC들이 낮의 관찰을 되새김
- **Phase 2**: 계획 수립 (Planning) - 다음 행동 계획
- **Phase 3**: 가족 회의 대화 (Family Meeting) - 3명이 함께 플레이어에 대해 논의
- **Phase 4**: 스탯 변화 분석 (Impact Analysis) - 호감도/의심도/인간성 격변
- **Phase 5**: 밤 나레이션 (Description) - 플레이어에게 보여줄 결과

In [9]:
!git clone https://github.com/DEUS-EX-MACHlNA/demo-repository.git
%cd demo-repository/
!git checkout dev/jinhyeok
!git log
!pwd

Cloning into 'demo-repository'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 231 (delta 121), reused 170 (delta 70), pack-reused 0 (from 0)
Receiving objects: 100% (231/231), 151.31 KiB | 16.81 MiB/s, done.
Resolving deltas: 100% (121/121), done.
/content/demo-repository/demo-repository/demo-repository
Branch 'dev/jinhyeok' set up to track remote branch 'dev/jinhyeok' from 'origin'.
Switched to a new branch 'dev/jinhyeok'
commit 981c9d104a8b56cdc464f0b2d49f106e4674a2d1 (HEAD -> dev/jinhyeok, origin/dev/jinhyeok)
Author: Jinhyeok33 <jinhyeok2844@naver.com>
Date:   Tue Feb 3 11:08:21 2026 +0900

    feat: Night Test served
    
    fix: LLM calling fixed

commit 53275c0657473e3706d6ead9fb1f07da17783a52
Merge: 30a06ef 5d5100e
Author: Jinhyeok33 <jinhyeok2844@naver.com>
Date:   Mon Feb 2 18:01:38 2026 +0900

    Merge from raphael

commit 30a06ef0cb3aede7202359ba404e11ff04901dee
Au

In [10]:
# from google.colab import userdata
# import os

# os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
from dotenv import load_dotenv
load_dotenv("../../.env")

False

## 0. Setup — 코렐라인 시나리오 로드 & WorldState 생성

In [11]:
import sys, logging
from pathlib import Path

# 프로젝트 루트를 sys.path에 추가
ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

logging.basicConfig(level=logging.INFO, format="%(name)s | %(message)s")
print(f"Project root: {ROOT}")

Project root: /content/demo-repository/demo-repository/demo-repository


In [ ]:
from app.loader import ScenarioLoader
from app.models import WorldState, NPCState

# 코렐라인 시나리오 로드
loader = ScenarioLoader(ROOT / "scenarios")
scenario_id = "coraline"  # 코렐라인 시나리오 사용
assets = loader.load(scenario_id)

print(f"시나리오: {assets.scenario.get('title')}")
print(f"NPC: {assets.get_all_npc_ids()}")
print(f"턴 제한: {assets.get_turn_limit()}")

In [ ]:
from app.agents.memory import MemoryEntry, add_memory, MEMORY_OBSERVATION

# ============================================================
# 테스트용 WorldState - 낮 동안 플레이어가 오브젝트를 만진 상황
# ============================================================
world = WorldState(
    turn=3,
    npcs={
        "button_mother": NPCState(npc_id="button_mother", trust=5, fear=0, suspicion=0, humanity=10),
        "button_father": NPCState(npc_id="button_father", trust=3, fear=0, suspicion=2, humanity=10),
        "button_daughter": NPCState(npc_id="button_daughter", trust=4, fear=0, suspicion=1, humanity=10),
    },
    inventory=[],
    vars={
        "humanity": 8,  # 플레이어 인간성
        "weapon_ready": 0,
        "family_eliminated": 0,
        "total_suspicion": 3,
        # 낮에 만진 오브젝트 로그
        "knife_touched": 1,      # 부엌칼을 만졌다!
        "match_touched": 1,      # 성냥을 만졌다!
        "needle_touched": 0,
        "mirror_touched": 1,     # 거울을 봤다
    },
)

# ============================================================
# 낮 동안의 관찰 기억 - 플레이어의 수상한 행동들
# ============================================================
day_observations = {
    "button_mother": [
        ("새 아이가 부엌을 서성거리더니 칼을 집어들었다.", 9.0),
        ("칼날을 빛에 비춰보며 뭔가를 확인하는 것 같았다.", 8.0),
        ("부엌에서 나올 때 표정이 묘하게 굳어있었다.", 7.0),
    ],
    "button_father": [
        ("새 아이가 창고에서 성냥갑을 발견했다.", 8.0),
        ("성냥을 호주머니에 넣으려다 눈이 마주쳤다.", 9.0),
        ("불안한 듯 시선을 피했다. 뭔가 숨기고 있다.", 7.0),
    ],
    "button_daughter": [
        ("새 언니/오빠가 깨진 거울을 한참 들여다봤어.", 6.0),
        ("거울에 비친 자기 눈을 보고 움찔했어.", 8.0),
        ("나한테 '여기서 나갈 수 있어?'라고 물었어. 왜 나가려는 거지?", 9.0),
    ],
}

for npc_id, obs_list in day_observations.items():
    for desc, imp in obs_list:
        entry = MemoryEntry.create(
            npc_id=npc_id, description=desc,
            importance_score=imp, current_turn=world.turn - 1,
            memory_type=MEMORY_OBSERVATION,
        )
        add_memory(world.npcs[npc_id].extras, entry)

# 성찰 트리거 - 모든 가족이 충분히 관찰했음
for npc_id in world.npcs:
    world.npcs[npc_id].extras["accumulated_importance"] = 25.0

print("=== 코렐라인 WorldState 준비 완료 ===")
print(f"플레이어 인간성: {world.vars['humanity']}")
print(f"총 의심도: {world.vars['total_suspicion']}")
print(f"\n[낮에 만진 오브젝트]")
print(f"  - 부엌칼: {'만짐' if world.vars['knife_touched'] else '안 만짐'}")
print(f"  - 성냥: {'만짐' if world.vars['match_touched'] else '안 만짐'}")
print(f"  - 거울: {'봄' if world.vars['mirror_touched'] else '안 봄'}")
print(f"\n[NPC 메모리]")
for npc_id, npc in world.npcs.items():
    stream = npc.extras.get("memory_stream", [])
    acc = npc.extras.get("accumulated_importance", 0)
    print(f"  {npc_id}: memories={len(stream)}, acc_importance={acc:.1f}")

In [14]:
from app.agents.llm import get_llm
from importlib import reload
import sys
reload(sys.modules["app.agents.llm"])

llm = get_llm()
print(f"LLM available: {llm.available}")
print("(available=False이면 fallback 로직으로 동작합니다)")

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

LLM available: True
(available=False이면 fallback 로직으로 동작합니다)


---
## Phase 1: 성찰 (Reflection)

In [15]:
from app.agents.reflection import should_reflect, perform_reflection


turn = world.turn
npc_ids = list(world.npcs.keys())
night_events: list[str] = []

print("=== Phase 1: 성찰 ===")
for npc_id in npc_ids:
    npc_state = world.npcs[npc_id]
    npc_data = assets.get_npc_by_id(npc_id)
    npc_name = npc_data["name"] if npc_data else npc_id
    persona = npc_data.get("persona", {}) if npc_data else {}

    trigger = should_reflect(npc_state.extras)
    print(f"\n[{npc_id}] should_reflect={trigger} (acc={npc_state.extras.get('accumulated_importance', 0):.1f})")

    if trigger:
        insights = perform_reflection(
            npc_id, npc_state.extras, npc_name, persona, llm, current_turn=turn,
        )
        print(f"  insights ({len(insights)}):\n" + "\n".join(f"    - {i}" for i in insights))
        night_events.append(f"{npc_name}이(가) 깊은 생각에 잠긴다.")
    else:
        print("  → 성찰 조건 미충족, skip")

print(f"\nnight_events: {night_events}")

ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 


=== Phase 1: 성찰 ===

[family] should_reflect=False (acc=18.0)
  → 성찰 조건 미충족, skip

[partner] should_reflect=False (acc=19.0)
  → 성찰 조건 미충족, skip

[witness] should_reflect=True (acc=45.0)
  insights (1):
    - 목격자은(는) 아직 답을 찾지 못했다.

night_events: ['목격자이(가) 깊은 생각에 잠긴다.']


---
## Phase 2: 계획 수립 (Planning)

In [ ]:
from app.agents.planning import update_plan

scenario_title = assets.scenario.get("title", "")
turn_limit = assets.get_turn_limit()

print("=== Phase 2: 계획 수립 ===")
for npc_id in npc_ids:
    npc_state = world.npcs[npc_id]
    npc_data = assets.get_npc_by_id(npc_id)
    npc_name = npc_data["name"] if npc_data else npc_id
    persona = npc_data.get("persona", {}) if npc_data else {}

    plan = update_plan(
        npc_id, npc_name, persona, npc_state.extras,
        npc_state.trust, npc_state.fear, npc_state.suspicion,
        turn, turn_limit, scenario_title, llm,
    )
    print(f"\n[{npc_id}] plan: {plan[:100]}..." if len(plan) > 100 else f"\n[{npc_id}] plan: {plan}")

ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 


=== Phase 2: 계획 수립 ===

[family] plan: 피해자 가족은(는) 다른 사람들과 대화를 나누어 보려 한다.

[partner] plan: 동료 수사관은(는) 다른 사람들과 대화를 나누어 보려 한다.

[witness] plan: 목격자은(는) 다른 사람들과 대화를 나누어 보려 한다.


## Phase 3: 가족 회의 대화 (Family Meeting - 3명이 함께)

In [ ]:
from app.agents.dialogue import generate_dialogue, store_dialogue_memories
from app.agents.utils import format_persona

# 가족 회의 - 3명이 함께 대화
all_conversations: list[tuple[str, str, list[dict[str, str]]]] = []

print("=== Phase 3: 가족 회의 ===")
print("단추 인형 가족이 밤에 모여 오늘 새 아이의 행동에 대해 논의합니다.\n")

# 회의 주제: 낮에 만진 오브젝트들
touched_objects = []
if world.vars.get("knife_touched"):
    touched_objects.append("부엌칼")
if world.vars.get("match_touched"):
    touched_objects.append("성냥")
if world.vars.get("mirror_touched"):
    touched_objects.append("거울")

print(f"[논의 대상 오브젝트]: {', '.join(touched_objects)}")
print("-" * 50)

# 가족 회의 시뮬레이션 (엄마-아빠, 엄마-딸 순차 대화로 구현)
meeting_pairs = [
    ("button_mother", "button_father"),
    ("button_mother", "button_daughter"),
]

for npc1_id, npc2_id in meeting_pairs:
    s1, s2 = world.npcs[npc1_id], world.npcs[npc2_id]
    d1 = assets.get_npc_by_id(npc1_id) or {}
    d2 = assets.get_npc_by_id(npc2_id) or {}

    print(f"\n--- {d1.get('name', npc1_id)} <-> {d2.get('name', npc2_id)} ---")

    conv = generate_dialogue(
        npc1_id, d1.get("name", npc1_id), d1.get("persona", {}), s1.extras,
        s1.trust, s1.fear, s1.suspicion,
        npc2_id, d2.get("name", npc2_id), d2.get("persona", {}), s2.extras,
        s2.trust, s2.fear, s2.suspicion,
        llm, current_turn=turn,
    )
    all_conversations.append((npc1_id, npc2_id, conv))

    for utt in conv:
        print(f"  {utt['speaker']}: {utt['text']}")

    # 대화 기억 저장
    p1_str = format_persona(d1.get("persona", {}))
    p2_str = format_persona(d2.get("persona", {}))
    store_dialogue_memories(
        npc1_id, d1.get("name", npc1_id), d2.get("name", npc2_id),
        conv, s1.extras, p1_str, llm, current_turn=turn,
    )
    store_dialogue_memories(
        npc2_id, d2.get("name", npc2_id), d1.get("name", npc1_id),
        conv, s2.extras, p2_str, llm, current_turn=turn,
    )

    night_events.append(
        f"{d1.get('name', npc1_id)}과(와) {d2.get('name', npc2_id)}이(가) 새 아이에 대해 이야기했다."
    )

print(f"\n총 회의 세션: {len(all_conversations)}")

## Phase 4: 스탯 변화 분석 (호감도/의심도/인간성 격변)

In [ ]:
from app.agents.dialogue import analyze_conversation_impact
from typing import Any

# 오브젝트 기반 직접 스탯 변화 계산
night_delta: dict[str, Any] = {
    "turn_increment": 1,
    "npc_stats": {},
    "vars": {},
}

print("=== Phase 4: 스탯 변화 분석 ===")
print("\n[오브젝트 터치에 따른 직접 영향]")

# 칼을 만졌으면 -> 의심도 급상승, 인간성 유지 (탈출 의지)
if world.vars.get("knife_touched"):
    print("  - 부엌칼 터치: 의심도 +3 (모든 NPC)")
    for npc_id in world.npcs:
        night_delta["npc_stats"].setdefault(npc_id, {})
        night_delta["npc_stats"][npc_id]["suspicion"] = night_delta["npc_stats"][npc_id].get("suspicion", 0) + 3

# 성냥을 만졌으면 -> 의심도 상승, 위험 인지
if world.vars.get("match_touched"):
    print("  - 성냥 터치: 의심도 +2 (모든 NPC), 총 의심도 +2")
    for npc_id in world.npcs:
        night_delta["npc_stats"].setdefault(npc_id, {})
        night_delta["npc_stats"][npc_id]["suspicion"] = night_delta["npc_stats"][npc_id].get("suspicion", 0) + 2
    night_delta["vars"]["total_suspicion"] = 2

# 거울을 봤으면 -> 인간성 회복, 호감도 하락 (그들과 다름을 인식)
if world.vars.get("mirror_touched"):
    print("  - 거울 봄: 인간성 +1, 호감도 -1 (모든 NPC)")
    night_delta["vars"]["humanity"] = 1
    for npc_id in world.npcs:
        night_delta["npc_stats"].setdefault(npc_id, {})
        night_delta["npc_stats"][npc_id]["trust"] = night_delta["npc_stats"][npc_id].get("trust", 0) - 1

# 대화에서 추가 영향 분석
print("\n[가족 회의 대화 영향]")
for npc1_id, npc2_id, conv in all_conversations:
    d1 = assets.get_npc_by_id(npc1_id) or {}
    d2 = assets.get_npc_by_id(npc2_id) or {}
    changes = analyze_conversation_impact(
        npc1_id, d1.get("name", npc1_id), d1.get("persona", {}),
        npc2_id, d2.get("name", npc2_id), d2.get("persona", {}),
        conv, llm,
    )
    print(f"\n  {d1.get('name')} <-> {d2.get('name')}:")
    for npc_id, stat_changes in changes.items():
        print(f"    {npc_id}: {stat_changes}")
        if stat_changes:
            night_delta["npc_stats"].setdefault(npc_id, {})
            for stat, val in stat_changes.items():
                night_delta["npc_stats"][npc_id][stat] = (
                    night_delta["npc_stats"][npc_id].get(stat, 0) + val
                )

print(f"\n[최종 night_delta]")
print(f"  npc_stats: {night_delta['npc_stats']}")
print(f"  vars: {night_delta['vars']}")

## Phase 5: 밤 나레이션 생성 (플레이어에게 보여줄 결과)

In [ ]:
from app.agents.generative_night import NightController

# 밤 나레이션 생성 - 가족 회의 결과를 플레이어에게 전달
night_description = NightController._generate_description(
    world, assets, all_conversations, night_events, llm,
)

print("=== Phase 5: 밤 나레이션 ===")
print("=" * 60)
print(night_description)
print("=" * 60)

# 추가: 의심도/호감도 변화 요약
print("\n[이번 밤의 변화 요약]")
for npc_id, stats in night_delta.get("npc_stats", {}).items():
    npc_data = assets.get_npc_by_id(npc_id) or {}
    npc_name = npc_data.get("name", npc_id)
    trust_change = stats.get("trust", 0)
    suspicion_change = stats.get("suspicion", 0)
    print(f"  {npc_name}: 호감도 {'+' if trust_change >= 0 else ''}{trust_change}, 의심도 {'+' if suspicion_change >= 0 else ''}{suspicion_change}")

humanity_change = night_delta.get("vars", {}).get("humanity", 0)
print(f"\n  [플레이어 인간성]: {'+' if humanity_change >= 0 else ''}{humanity_change}")

---
## Final: NightResult 조립

In [ ]:
from app.models import NightResult

night_conversation: list[list[dict[str, str]]] = [
    conv for _, _, conv in all_conversations
]

result = NightResult(
    night_delta=night_delta,
    night_conversation=night_conversation,
    night_description=night_description,
)

print("=== NightResult ===")
print(f"night_delta: {result.night_delta}")
print(f"\nnight_description:\n  {result.night_description}")
print(f"\nnight_conversation ({len(result.night_conversation)} rounds):")
for i, conv in enumerate(result.night_conversation):
    print(f"\n  [Round {i+1}] ({len(conv)} utterances)")
    for utt in conv:
        print(f"    {utt['speaker']}: {utt['text']}")

=== NightResult ===
night_delta: {'turn_increment': 1, 'npc_stats': {}, 'vars': {}}

night_description:
  밤이 깊어간다. 진실과 조작의 경계가 흐려진다. 목격자이(가) 깊은 생각에 잠긴다.

night_conversation (2 rounds):

  [Round 1] (4 utterances)
    피해자 가족: ...피해자 가족은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.
    피해자 가족: ...피해자 가족은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.

  [Round 2] (4 utterances)
    동료 수사관: ...동료 수사관은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.
    동료 수사관: ...동료 수사관은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.


## (참고) NightController.run() 통합 실행 - 코렐라인 버전

In [ ]:
from app.agents.generative_night import NightController
from app.models import WorldState, NPCState
from app.agents.memory import MemoryEntry, add_memory, MEMORY_OBSERVATION

# 새 WorldState (다른 오브젝트 조합 테스트)
world2 = WorldState(
    turn=5,
    npcs={
        "button_mother": NPCState(npc_id="button_mother", trust=7, fear=0, suspicion=1, humanity=10),
        "button_father": NPCState(npc_id="button_father", trust=5, fear=0, suspicion=3, humanity=10),
        "button_daughter": NPCState(npc_id="button_daughter", trust=6, fear=0, suspicion=2, humanity=10),
    },
    inventory=["sewing_needle"],  # 바늘 획득
    vars={
        "humanity": 6,  # 인간성 감소 중
        "weapon_ready": 1,  # 무기 준비됨
        "family_eliminated": 0,
        "total_suspicion": 6,
        "knife_touched": 0,
        "match_touched": 0,
        "needle_touched": 1,  # 바늘만 만짐
        "mirror_touched": 0,
    },
)

# 다른 관찰 기억
day_observations_2 = {
    "button_mother": [
        ("새 아이가 재봉 바구니를 뒤지더니 바늘을 가져갔다.", 9.0),
        ("바늘을 옷 안쪽에 숨기는 것을 봤다.", 10.0),
    ],
    "button_father": [
        ("새 아이가 창문 쪽을 계속 쳐다봤다. 나가고 싶은 거다.", 8.0),
        ("규칙을 어기고 있다. 처벌이 필요하다.", 9.0),
    ],
    "button_daughter": [
        ("새 언니/오빠가 나랑 안 놀아줬어. 싫어하는 거야.", 7.0),
        ("엄마한테 다 말해야겠어.", 8.0),
    ],
}

for npc_id, obs_list in day_observations_2.items():
    for desc, imp in obs_list:
        entry = MemoryEntry.create(
            npc_id=npc_id, description=desc,
            importance_score=imp, current_turn=4,
            memory_type=MEMORY_OBSERVATION,
        )
        add_memory(world2.npcs[npc_id].extras, entry)

for npc_id in world2.npcs:
    world2.npcs[npc_id].extras["accumulated_importance"] = 35.0

controller = NightController()
result2 = controller.run(world2, assets)

print("=== 통합 실행 결과 ===")
print(f"night_delta: {result2.night_delta}")
print(f"\nnight_description:\n{result2.night_description}")
print(f"\nconversation rounds: {len(result2.night_conversation)}")